### Importing packages ###

In [1]:
import typing
from typing import Union
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, SamplingVQE, QAOA
# from qiskit.primitives import Sampler
from qiskit_optimization.converters import LinearEqualityToPenalty
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.translators import from_docplex_mp
from qiskit.utils import algorithm_globals
# from qiskit_ibm_runtime import QiskitRuntimeService, Options, Session, Sampler
import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model
algorithm_globals.random_seed = 123456
# from qiskit.algorithms.minimum_eigensolvers import QAOA
# from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_aer.primitives import Sampler
from qiskit.quantum_info import Pauli, Statevector
from qiskit.result import QuasiDistribution
from qiskit.utils import algorithm_globals
import random

### Loading the data ###

In [2]:
import yfinance as yahooFinance
import pandas as pd
df = pd.read_csv("final_ticker_list - tickers_final.csv")
ticker_list = []
for i in range(466):
    ticker_list.append(df.iat[i, 0])
new_data = yahooFinance.download(tickers = ticker_list, period = "3mo")
working_data = pd.DataFrame(new_data['Adj Close'])

[*********************100%***********************]  466 of 466 completed

3 Failed downloads:
- CASTROLIND.: No data found, symbol may be delisted
- ADANITRANS-BL.NS: No data found, symbol may be delisted
- CARBORUNI.NS: No data found for this date range, symbol may be delisted


In [3]:
working_data.head(10)

,360ONE.NS,3MINDIA.NS,AARTIPHARM.NS,ABB.NS,ABBOTINDIA.NS,ABFRL.NS,ACC.NS,ACE.NS,ACI.NS,ADANIGREEN.NS,...,WELSPUNIND.NS,WESTLIFE.NS,WHEELS.NS,WHIRLPOOL.NS,WIPRO.NS,ZEEL.NS,ZENSARTECH.NS,ZFCVINDIA.NS,ZOMATO.NS,ZYDUSLIFE.NS
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-06 00:00:00,442.166138,23406.300781,305.250000,3313.082275,20575.550781,236.649994,1865.800049,368.850006,644.099976,589.799988,...,69.099998,668.599976,513.599976,1339.699951,393.750000,195.399994,280.649994,10648.650391,53.950001,476.149994
2023-03-08 00:00:00,437.908813,23627.300781,301.149994,3374.383057,20615.400391,235.850006,1886.500000,381.850006,630.400024,619.250000,...,70.000000,677.000000,522.700012,1363.400024,392.600006,196.649994,277.700012,10406.099609,54.799999,471.450012
2023-03-09 00:00:00,435.730682,23536.199219,299.100006,3382.519775,20680.150391,235.050003,1860.099976,377.450012,621.650024,650.200012,...,68.349998,671.950012,520.250000,1363.650024,390.100006,193.800003,282.950012,10470.500000,54.049999,468.700012
2023-03-10 00:00:00,438.007812,23582.800781,303.000000,3371.936768,20554.650391,235.000000,1846.900024,380.799988,635.900024,682.700012,...,69.099998,670.250000,511.500000,1354.400024,389.100006,193.600006,274.399994,10195.250000,53.849998,474.899994
2023-03-13 00:00:00,425.928955,23540.099609,300.299988,3269.153320,20263.599609,227.800003,1770.000000,388.649994,641.000000,716.799988,...,67.650002,671.450012,495.299988,1315.150024,385.450012,188.300003,268.549988,10166.849609,51.900002,466.200012
2023-03-14 00:00:00,419.443970,23103.150391,298.000000,3305.793945,20329.000000,218.149994,1738.400024,375.399994,639.500000,706.099976,...,66.900002,663.099976,497.850006,1324.650024,379.450012,192.449997,267.149994,9931.849609,50.750000,470.549988
2023-03-15 00:00:00,428.008087,23371.099609,302.149994,3296.908447,20367.500000,214.750000,1740.000000,371.850006,639.400024,740.400024,...,66.849998,669.950012,501.450012,1339.800049,375.750000,189.050003,267.899994,10160.450195,50.750000,473.399994
2023-03-16 00:00:00,428.800171,22982.449219,291.100006,3322.566895,20704.000000,215.000000,1750.300049,361.049988,629.849976,777.400024,...,64.750000,682.150024,500.200012,1340.900024,372.850006,206.699997,268.100006,10192.150391,53.049999,479.649994
2023-03-17 00:00:00,420.186523,22936.349609,285.899994,3307.191895,20928.300781,215.050003,1730.400024,377.700012,622.250000,816.250000,...,65.300003,687.750000,502.149994,1363.000000,376.500000,207.250000,277.500000,10178.500000,54.900002,477.350006


### Step 1 :
We would scale each stock closing prices between 0 and 1 through a simple linear approach. We also drop tickers that have missing values. We could later focus on treatments for missing values

In [4]:
def data_process(df):
    ts_na=df.isna().sum()

    wd2=df.drop(columns=list(ts_na[ts_na!=0].index))

    wd2 -= wd2.min()  # equivalent to df = df - df.min()
    wd2 /= wd2.max()

    return wd2

In [5]:
wd2=data_process(working_data)

In [6]:
wd2.head(10)

,360ONE.NS,3MINDIA.NS,AARTIPHARM.NS,ABB.NS,ABBOTINDIA.NS,ABFRL.NS,ACC.NS,ACE.NS,ACI.NS,ADANIGREEN.NS,...,WELSPUNIND.NS,WESTLIFE.NS,WHEELS.NS,WHIRLPOOL.NS,WIPRO.NS,ZEEL.NS,ZENSARTECH.NS,ZFCVINDIA.NS,ZOMATO.NS,ZYDUSLIFE.NS
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-06 00:00:00,1.000000,0.287745,0.225178,0.168643,0.119186,1.000000,0.924051,0.057842,0.888846,0.000000,...,0.164706,0.032005,0.356672,0.319747,0.736997,0.444892,0.128986,0.517378,0.173034,0.167508
2023-03-08 00:00:00,0.904972,0.326927,0.194455,0.231342,0.134411,0.982906,1.000000,0.154246,0.784186,0.067000,...,0.188235,0.080885,0.406645,0.486297,0.714426,0.478494,0.108079,0.342308,0.211236,0.088384
2023-03-09 00:00:00,0.856354,0.310775,0.179093,0.239665,0.159150,0.965812,0.903137,0.121617,0.717342,0.137413,...,0.145098,0.051499,0.393191,0.488054,0.665358,0.401882,0.145287,0.388791,0.177528,0.042088
2023-03-10 00:00:00,0.907182,0.319037,0.208318,0.228840,0.111201,0.964744,0.854706,0.146459,0.826203,0.211353,...,0.164706,0.041606,0.345140,0.423050,0.645731,0.396505,0.084692,0.190119,0.168539,0.146464
2023-03-13 00:00:00,0.637569,0.311467,0.188085,0.123713,0.000000,0.810898,0.572556,0.204672,0.865164,0.288932,...,0.126797,0.048589,0.256178,0.147225,0.574092,0.254032,0.043232,0.169620,0.080899,0.000000
2023-03-14 00:00:00,0.492817,0.233997,0.170850,0.161189,0.024987,0.604701,0.456614,0.106415,0.853705,0.264589,...,0.107190,0.000000,0.270181,0.213985,0.456330,0.365591,0.033310,0.000000,0.029214,0.073232
2023-03-15 00:00:00,0.683977,0.281503,0.201948,0.152101,0.039697,0.532051,0.462484,0.080089,0.852941,0.342623,...,0.105882,0.039861,0.289951,0.320450,0.383709,0.274193,0.038625,0.165001,0.029214,0.121212
2023-03-16 00:00:00,0.701658,0.212597,0.119146,0.178344,0.168262,0.537393,0.500275,0.000000,0.779984,0.426800,...,0.050980,0.110853,0.283086,0.328180,0.326791,0.748656,0.040042,0.187882,0.132584,0.226431
2023-03-17 00:00:00,0.509392,0.204424,0.080180,0.162619,0.253960,0.538462,0.427261,0.123471,0.721925,0.515186,...,0.065360,0.143439,0.293795,0.483486,0.398430,0.763441,0.106662,0.178029,0.215730,0.187710


### Step 2: 
Suppose there are 500 stocks and we only can use 5 qubits (we might be able to use more. Maybe, we can try for a paid IBM service later on for IBM Falcon with 27 qubits, but that seems expensive: 1.6 USD/sec). We will randomly divide our portfolio into sub-portfolios with 5 stocks each. This would give us 100 sub-portfolios.

In [7]:
def partition(lst, n): 
    """
    lst: list to be partitioned
    n: number of partitions
    """
    division = len(lst) / float(n) 
    return [ lst[int(round(division * i)): int(round(division * (i + 1)))] for i in range(n) ]

In [8]:
#EXAMPLE below. In the algorithm, we would first shuffle the columns and use this partition function so that we are randomly 
# partitioning our portfolio
partition(list(range(10)),3)

[[0, 1, 2], [3, 4, 5, 6], [7, 8, 9]]

### We need to run QAOA several times and so we will create a function that would give us QAOA results ###

In [9]:
def callback(i, params, obj, stddev, alpha):
    # we translate the objective from the internal Ising representation
    # to the original optimization problem 
    objectives[alpha].append(np.real_if_close(-(obj + offset)))

#BELOW IS USED TO CONVERT THE BEST MEASUREMENT OUTPUT FROM QAOA INTO A LIST OF Os and 1s.    
def bitfield(n: int, L: int) -> list[int]:
    result = np.binary_repr(n, L)
    return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

# WE DO NOT USE THE BELOW FUNCTION (sample_most_likely) BUT I HAVE KEPT IT JUST IN CASE IT WILL BE NEEDED LATER #
def sample_most_likely(state_vector: Union[QuasiDistribution, Statevector]) -> np.ndarray:
    """Compute the most likely binary string from state vector.
    Args:
        state_vector: State vector or quasi-distribution.

    Returns:
        Binary string as an array of ints.
    """
    if isinstance(state_vector, QuasiDistribution):
        values = list(state_vector.values())
    else:
        values = state_vector
    n = int(np.log2(len(values)))
    k = np.argmax(np.abs(values))
    x = bitfield(k, n)
    x.reverse()
    return np.asarray(x)

In [10]:
#BELOW IS THE USUSAL QAOA IMPLEMENTATION< THERE IS A CALLBACK FUNCTION DEFINED IN THE ABOVE CELL, THE QAOA BEST MEASUERMENT
# OUTPUT IS THE FORM OF A STRING, SO bitfield helps us get it in the form of list of 0s and 1s, as desired. I have set the
# CVaR alpha to 0.75
def qaoa_run(mu,sigma,n:int,q:float,budget:int,penalty):
    
    """
    n: number of assets
    q: risk factor
    budget: budget
    penalty: 2 * n  # scaling of penalty term
    
    """
    mdl = Model("portfolio_optimization")
    x = mdl.binary_var_list(range(n), name="x")
    objective = mdl.sum([mu[i] * x[i] for i in range(n)])
    objective -= q * mdl.sum([sigma[i, j] * x[i] * x[j] for i in range(n) for j in range(n)])
    mdl.maximize(objective)
    mdl.add_constraint(mdl.sum(x[i] for i in range(n)) == budget)
    qp = from_docplex_mp(mdl)
    
    linear2penalty = LinearEqualityToPenalty(penalty=penalty)
    qp = linear2penalty.convert(qp)
    global _
    global offset
    _, offset = qp.to_ising()
    
    maxiter = 100
    optimizer = COBYLA(maxiter=maxiter)

#     ansatz = RealAmplitudes(n, reps=1)
#     m = ansatz.num_parameters

    sampler = Sampler() #currently using a simulator
    # we might use a noisy simulator eventually and also use qiskit runtime later

    global alphas
    alphas = [0.75]  # confidence levels to be evaluated
    # dictionaries to store optimization progress and results
    global objectives
    objectives = {alpha: [] for alpha in alphas}  # set of tested objective functions w.r.t. alpha
    results = {}  # results of minimum eigensolver w.r.t alpha

    for alpha in alphas:

        # initialize SamplingVQE using CVaR
        qaoa = QAOA(
            sampler=sampler,
            optimizer=optimizer,
            aggregation=alpha,
            callback=lambda i, params, obj, stddev: callback(i, params, obj, stddev, alpha),
        )

        qubit_op=_.primitive
        result = qaoa.compute_minimum_eigenvalue(qubit_op)
        print(result)
        print(result.best_measurement['state'])
        x=bitfield(result.best_measurement['state'],n)
        return x

In [11]:
#LET'S RUN ABOVE WITH AN EXAMPLE, I WILL RUN IT WITH 6 STOCKS, I have taken this from qiskit's portfolio optimization 
# tutorial, WE PRINT OUT THE RESULT AND THE BEST MEASUREMENT OUTPUT FROM QAOA, which is in an int. To convert that to bits
# we used the bitfield function
mu = np.array([0.7313, 0.9893, 0.2725, 0.8750, 0.7667, 0.3622])
sigma = np.array(
    [
        [0.7312, -0.6233, 0.4689, -0.5452, -0.0082, -0.3809],
        [-0.6233, 2.4732, -0.7538, 2.4659, -0.0733, 0.8945],
        [0.4689, -0.7538, 1.1543, -1.4095, 0.0007, -0.4301],
        [-0.5452, 2.4659, -1.4095, 3.5067, 0.2012, 1.0922],
        [-0.0082, -0.0733, 0.0007, 0.2012, 0.6231, 0.1509],
        [-0.3809, 0.8945, -0.4301, 1.0922, 0.1509, 0.8992],
    ]
)

n=6
q=0.2
budget = n // 2
penalty=2*n
qaoa_run(mu=mu,sigma=sigma,n=6,q=0.2,budget=3,penalty=2*6)

SamplingMinimumEigensolverResult:
	Eigenvalue: -12.409480911458328
	Best measurement
: {'state': 19, 'bitstring': '010011', 'value': (-19.048989999999996+0j), 'probability': 0.0390625}

19


[0, 1, 0, 0, 1, 1]

### Step 3: 

Sub-steps:
1. Since we are using only 5 qubits.
2. The plan is to divide the portfolio into sub-portfolios of size<=5, we might end up having some sub-portfolios of size 4
3. We created a utility function called partition which would help us. The partition function would only divide groups sequentially, but we want to randomly create groups. To do this, we will first shuffle our dataframe columns itself.
4. We will run QAOA using qaoa_run on each of the sub-portfolios. For say a dataframe of 458 stocks, we can expect to have int(458/5)+1 sub-portfolios, which is 92.
5. But Since 92>5, we would have to do some merging now. We will treat a single sub-portfolio as 1 node or time series by taking it as a daily means of all constituent stocks. We will have int(92/5)+1 = 19 bigger sub-portfolios, each having the smaller sub-portfolios represented as combined time series. The mu and sigma for qaoa would be taken for these new time series in the bigger sub-portfolios.We will run qaoas for the bigger 19 sub-portfolios.
6. 19>5, so we will repeat and now we will have int(19/5)+1=4 even bigger subportfolios and again create new time series and run qaoas on these 4. 4<5, so we can run qaoa on a portfolio of new time series (4 of them) and stop there.
7. We need to finally combine all qaoa results. We do this by going backwards, for a 0 in one qaoa result, the contstituents of the corresponding sub-portfolio would have all its qaoa results in the previous step negated.

In [40]:
#THIS FUNCTION CARRIES OUT ALL THE STEPS LISTED ABOVE AND RETURNS THE LIST OF STOCKS DETECTED AS WORTH INVESTING IN BY OUR 
# ALGORITHM
def qaoa_in_qaoa(df,nqubits):
    n=len(df.columns)
    c=0
    part_num=[]
    while(n>nqubits):
        part_num.append(n)
        n=int(n/nqubits)+1   
        c+=1
    part_num.append(n)
    
    main_li=list(range(len(df.columns)))
    random.shuffle(main_li)
    partitions_ind=[]
    for i in range(len(part_num)-1):
        partitions_ind.append(partition(list(range(part_num[i])),part_num[i+1]))
         
    part_df=[]
    part_df.append(df)
    for j in range(len(partitions_ind)):
        x=pd.concat([part_df[j].iloc[:,partitions_ind[j][i]].mean(axis=1) for i in range(len(partitions_ind[j]))], axis=1)
        part_df.append(x)

    res_sub_qaoa=[]
    for i in range(len(part_df)-1):
        res = []
        for j in partitions_ind[i]:
            mu = np.array(part_df[i].iloc[:,j].mean().values)
            sigma = np.array(part_df[i].iloc[:,j].cov().values)
            x= qaoa_run(mu=mu,sigma=sigma,n=mu.shape[0],q=0.5,budget=3,penalty=2*mu.shape[0])
            res.append(list(x))
        res_sub_qaoa.append(res)
    mu_final=np.array(part_df[-1].mean().values)
    sig_final=np.array(part_df[-1].cov().values)
    x=qaoa_run(mu=mu_final,sigma=sig_final,n=mu_final.shape[0],q=0.5,budget=3,penalty=2*mu_final.shape[0])
    res_sub_qaoa.append(list(x))
    
    temp=[]
    temp.append(res_sub_qaoa[-1])
    res_sub_qaoa[-1]=temp
    
    temp=[]
    temp.append(list(range(len(res_sub_qaoa[-1][0]))))
    partitions_ind.append(temp)
    
#     res_tr=res_sub_qaoa
    for i in range(1,len(res_sub_qaoa))[::-1]:
        for j in range(len(res_sub_qaoa[i])):
            for k in range(len(res_sub_qaoa[i][j])):
                if res_sub_qaoa[i][j][k]==0:
                    ind_zero=partitions_ind[i][j][k]
#                     print(ind_zero)
#                     print(partitions_ind[i-1][ind_zero])
#                     print(res_sub_qaoa[i-1][ind_zero])
                    z=res_sub_qaoa[i-1][ind_zero]
                    z1=[abs(x-1) for x in z]
#                     print(z1)
                    res_sub_qaoa[i-1][ind_zero]=z1   
                    
    fin_res = [item for sublist in res_sub_qaoa[0] for item in sublist] 
    fin_ind = [item for sublist in partitions_ind[0] for item in sublist]
    ind_1 = [i for i, j in enumerate(fin_res) if j == 1]
    main_ind1= [fin_ind[i] for i in ind_1]
    
    return list(wd2.columns[main_ind1])

In [42]:
final_res=qaoa_in_qaoa(wd2,5)

SamplingMinimumEigensolverResult:
	Eigenvalue: -14.81528707702001
	Best measurement
: {'state': 21, 'bitstring': '10101', 'value': (-15.407097936767052+0j), 'probability': 0.099609375}

21
SamplingMinimumEigensolverResult:
	Eigenvalue: -6.2993473614801
	Best measurement
: {'state': 19, 'bitstring': '10011', 'value': (-15.447317528534267+0j), 'probability': 0.025390625}

19
SamplingMinimumEigensolverResult:
	Eigenvalue: -14.94033503953587
	Best measurement
: {'state': 19, 'bitstring': '10011', 'value': (-15.306138038149054+0j), 'probability': 0.0849609375}

19
SamplingMinimumEigensolverResult:
	Eigenvalue: -14.735652175235197
	Best measurement
: {'state': 25, 'bitstring': '11001', 'value': (-15.371226010558143+0j), 'probability': 0.0859375}

25
SamplingMinimumEigensolverResult:
	Eigenvalue: -11.938281869375048
	Best measurement
: {'state': 7, 'bitstring': '00111', 'value': (-15.338947630194316+0j), 'probability': 0.0693359375}

7
SamplingMinimumEigensolverResult:
	Eigenvalue: -9.5942187

SamplingMinimumEigensolverResult:
	Eigenvalue: -9.791354506659717
	Best measurement
: {'state': 21, 'bitstring': '10101', 'value': (-15.415681010632223+0j), 'probability': 0.0595703125}

21
SamplingMinimumEigensolverResult:
	Eigenvalue: -9.967292112487101
	Best measurement
: {'state': 28, 'bitstring': '11100', 'value': (-15.33057745743954+0j), 'probability': 0.0537109375}

28
SamplingMinimumEigensolverResult:
	Eigenvalue: -9.310426389224583
	Best measurement
: {'state': 7, 'bitstring': '00111', 'value': (-15.361460271071024+0j), 'probability': 0.0419921875}

7
SamplingMinimumEigensolverResult:
	Eigenvalue: -14.764790208843381
	Best measurement
: {'state': 7, 'bitstring': '00111', 'value': (-15.379502930723124+0j), 'probability': 0.07421875}

7
SamplingMinimumEigensolverResult:
	Eigenvalue: -12.769099957986771
	Best measurement
: {'state': 26, 'bitstring': '11010', 'value': (-15.299809370123665+0j), 'probability': 0.0595703125}

26
SamplingMinimumEigensolverResult:
	Eigenvalue: -13.7506

SamplingMinimumEigensolverResult:
	Eigenvalue: -11.788333718249158
	Best measurement
: {'state': 13, 'bitstring': '01101', 'value': (-15.318700940725332+0j), 'probability': 0.0673828125}

13
SamplingMinimumEigensolverResult:
	Eigenvalue: -15.077941931966127
	Best measurement
: {'state': 28, 'bitstring': '11100', 'value': (-15.354161487970702+0j), 'probability': 0.087890625}

28
SamplingMinimumEigensolverResult:
	Eigenvalue: -8.881337546225463
	Best measurement
: {'state': 21, 'bitstring': '10101', 'value': (-15.258268460954078+0j), 'probability': 0.033203125}

21
SamplingMinimumEigensolverResult:
	Eigenvalue: -9.996093583984173
	Best measurement
: {'state': 25, 'bitstring': '11001', 'value': (-15.478716112996466+0j), 'probability': 0.0478515625}

25
SamplingMinimumEigensolverResult:
	Eigenvalue: -15.082118137561618
	Best measurement
: {'state': 14, 'bitstring': '01110', 'value': (-15.30222331793235+0j), 'probability': 0.091796875}

14
SamplingMinimumEigensolverResult:
	Eigenvalue: -10.

In [43]:
final_res

['3MINDIA.NS',
 'ABB.NS',
 'ABFRL.NS',
 'ACI.NS',
 'ADANIGREEN.NS',
 'ADANIPORTS.NS',
 'AFFLE.NS',
 'AGSTRA.NS',
 'AHLUCONT.NS',
 'AIAENG.NS',
 'ALICON.NS',
 'ALKEM.NS',
 'ALOKINDS.NS',
 'AMRUTANJAN.NS',
 'ANUP.NS',
 'ANURAS.NS',
 'APLLTD.NS',
 'APOLLOHOSP.NS',
 'APOLLOTYRE.NS',
 'ARIHANTSUP.NS',
 'ASIANPAINT.NS',
 'AUBANK.NS',
 'AUROPHARMA.NS',
 'AVANTIFEED.NS',
 'BAJAJELEC.NS',
 'BALAMINES.NS',
 'BALKRISIND.NS',
 'BALRAMCHIN.NS',
 'BATAINDIA.NS',
 'BDL.NS',
 'BECTORFOOD.NS',
 'BEML.NS',
 'BERGEPAINT.NS',
 'BIOCON.NS',
 'BLUESTARCO.NS',
 'BSOFT.NS',
 'CAMLINFINE.NS',
 'CAMS.NS',
 'CANFINHOME.NS',
 'CCL.NS',
 'CEATLTD.NS',
 'CENTURYPLY.NS',
 'CENTURYTEX.NS',
 'CHEMPLASTS.NS',
 'CHOLAFIN.NS',
 'CIPLA.NS',
 'COLPAL.NS',
 'CONCOR.NS',
 'CREDITACC.NS',
 'CROMPTON.NS',
 'CUMMINSIND.NS',
 'DABUR.NS',
 'DATAPATTNS.NS',
 'DCXINDIA.NS',
 'DEEPAKFERT.NS',
 'DEEPAKNTR.NS',
 'DHANUKA.NS',
 'DLF.NS',
 'DRREDDY.NS',
 'DWARKESH.NS',
 'DYNAMATECH.NS',
 'EICHERMOT.NS',
 'ELECON.NS',
 'ERIS.NS',
 'FEDER

In [44]:
len(final_res)

231

### END ###

In [26]:
i

2

In [27]:
j

[14, 15, 16, 17, 18]

In [29]:
# k

In [17]:
df=wd2.copy()
nqubits=5
n=len(df.columns)
c=0
part_num=[]
while(n>nqubits):
    part_num.append(n)
    n=int(n/nqubits)+1   
    c+=1
part_num.append(n)

main_li=list(range(len(df.columns)))
random.shuffle(main_li)
partitions_ind=[]
for i in range(len(part_num)-1):
    partitions_ind.append(partition(list(range(part_num[i])),part_num[i+1]))

part_df=[]
part_df.append(df)
for j in range(len(partitions_ind)):
    x=pd.concat([part_df[j].iloc[:,partitions_ind[j][i]].mean(axis=1) for i in range(len(partitions_ind[j]))], axis=1)
    part_df.append(x)

res_sub_qaoa=[]
for i in range(len(part_df)-1):
    res = []
    for j in partitions_ind[i]:
        mu = np.array(part_df[i].iloc[:,j].mean().values)
        sigma = np.array(part_df[i].iloc[:,j].cov().values)
        x= qaoa_run(mu=mu,sigma=sigma,n=mu.shape[0],q=0.5,budget=3,penalty=2*mu.shape[0])
        res.append(list(x))
    res_sub_qaoa.append(res)
mu_final=np.array(part_df[-1].mean().values)
sig_final=np.array(part_df[-1].cov().values)
x=qaoa_run(mu=mu_final,sigma=sig_final,n=mu_final.shape[0],q=0.5,budget=3,penalty=2*mu_final.shape[0])
res_sub_qaoa.append(list(x))

SamplingMinimumEigensolverResult:
	Eigenvalue: -10.9656905652893
	Best measurement
: {'state': 21, 'bitstring': '10101', 'value': (-15.407097936767052+0j), 'probability': 0.0517578125}

21
SamplingMinimumEigensolverResult:
	Eigenvalue: -13.168175485227703
	Best measurement
: {'state': 19, 'bitstring': '10011', 'value': (-15.447317528534267+0j), 'probability': 0.0810546875}

19
SamplingMinimumEigensolverResult:
	Eigenvalue: -15.04434729948435
	Best measurement
: {'state': 19, 'bitstring': '10011', 'value': (-15.306138038149054+0j), 'probability': 0.0888671875}

19
SamplingMinimumEigensolverResult:
	Eigenvalue: -15.211721996133871
	Best measurement
: {'state': 25, 'bitstring': '11001', 'value': (-15.371226010558143+0j), 'probability': 0.099609375}

25
SamplingMinimumEigensolverResult:
	Eigenvalue: -12.287629565826805
	Best measurement
: {'state': 7, 'bitstring': '00111', 'value': (-15.338947630194316+0j), 'probability': 0.0673828125}

7
SamplingMinimumEigensolverResult:
	Eigenvalue: -15.

SamplingMinimumEigensolverResult:
	Eigenvalue: -12.318557597855875
	Best measurement
: {'state': 21, 'bitstring': '10101', 'value': (-15.415681010632223+0j), 'probability': 0.0791015625}

21
SamplingMinimumEigensolverResult:
	Eigenvalue: -5.4417461392457485
	Best measurement
: {'state': 28, 'bitstring': '11100', 'value': (-15.33057745743954+0j), 'probability': 0.0703125}

28
SamplingMinimumEigensolverResult:
	Eigenvalue: -12.063803674139132
	Best measurement
: {'state': 7, 'bitstring': '00111', 'value': (-15.361460271071024+0j), 'probability': 0.072265625}

7
SamplingMinimumEigensolverResult:
	Eigenvalue: -12.960077533639126
	Best measurement
: {'state': 7, 'bitstring': '00111', 'value': (-15.379502930723124+0j), 'probability': 0.068359375}

7
SamplingMinimumEigensolverResult:
	Eigenvalue: -10.769971545230515
	Best measurement
: {'state': 26, 'bitstring': '11010', 'value': (-15.299809370123665+0j), 'probability': 0.0498046875}

26
SamplingMinimumEigensolverResult:
	Eigenvalue: -13.9437

SamplingMinimumEigensolverResult:
	Eigenvalue: -13.406248245314899
	Best measurement
: {'state': 13, 'bitstring': '01101', 'value': (-15.318700940725332+0j), 'probability': 0.078125}

13
SamplingMinimumEigensolverResult:
	Eigenvalue: -14.793113839449017
	Best measurement
: {'state': 28, 'bitstring': '11100', 'value': (-15.354161487970702+0j), 'probability': 0.06640625}

28
SamplingMinimumEigensolverResult:
	Eigenvalue: -13.638721611991397
	Best measurement
: {'state': 21, 'bitstring': '10101', 'value': (-15.258268460954078+0j), 'probability': 0.0771484375}

21
SamplingMinimumEigensolverResult:
	Eigenvalue: -13.79150771024442
	Best measurement
: {'state': 25, 'bitstring': '11001', 'value': (-15.478716112996466+0j), 'probability': 0.0791015625}

25
SamplingMinimumEigensolverResult:
	Eigenvalue: -12.981002180774809
	Best measurement
: {'state': 14, 'bitstring': '01110', 'value': (-15.30222331793235+0j), 'probability': 0.083984375}

14
SamplingMinimumEigensolverResult:
	Eigenvalue: -13.634

In [18]:
temp=[]
temp.append(res_sub_qaoa[-1])
res_sub_qaoa[-1]=temp

In [19]:
res_sub_qaoa[3]

[[0, 1, 1, 1]]

In [21]:
res_sub_qaoa[2][:3]

[[0, 1, 0, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 1]]

In [25]:
partitions_ind[2]

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13], [14, 15, 16, 17, 18]]

In [30]:
for i in range(1,len(res_sub_qaoa))[::-1]:
    for j in range(len(res_sub_qaoa[i])):
        for k in range(len(res_sub_qaoa[i][j])):
            if res_sub_qaoa[i][j][k]==0:
                ind_zero=partitions_ind[i][j][k]
#                     print(ind_zero)
#                     print(partitions_ind[i-1][ind_zero])
#                     print(res_sub_qaoa[i-1][ind_zero])
                z=res_sub_qaoa[i-1][ind_zero]
                z1=[abs(x-1) for x in z]
#                     print(z1)
                res_sub_qaoa[i-1][ind_zero]=z1   

IndexError: list index out of range

In [34]:
temp=[]
temp.append(list(range(len(res_sub_qaoa[-1][0]))))

In [38]:
partitions_ind.append(temp)

In [39]:
partitions_ind[3]

[[0, 1, 2, 3]]

In [32]:
i,j,k

(3, 0, 0)

In [33]:
partitions_ind[3]

IndexError: list index out of range

In [14]:
len(res_sub_qaoa)

4

In [15]:
len(res_sub_qaoa[0])

92

In [20]:
len(res_sub_qaoa[1])

19

In [21]:
len(res_sub_qaoa[2])

4

In [22]:
len(res_sub_qaoa[3])

4

In [26]:
res_sub_qaoa[3]

[0, 1, 1, 1]

In [24]:
res_sub_qaoa[2][1]

[0, 1, 1, 0, 1]

In [29]:
temp=[]

temp.append(res_sub_qaoa[3])

In [30]:
temp

[[0, 1, 1, 1]]

In [32]:
res_sub_qaoa[-1]=temp

In [34]:
res_sub_qaoa[0][3]

[1, 1, 0, 0, 1]

In [36]:
res_sub_qaoa[1][2]

[1, 1, 0, 0, 1]

In [37]:
res_sub_qaoa[2][3]

[0, 1, 1, 1, 0]

In [38]:
res_sub_qaoa[3]

[[0, 1, 1, 1]]